# Standardize

Pull together our different data sources and combine them into a single dataset.

Import Python tools

In [4]:
import pandas as pd

## Department of Streets and Sanitation

Read in the raw data

In [5]:
dss_df = pd.read_excel(
    "input/FOIA_Trees_Planted_2011_thru_2021.xls",
    skipfooter=2,  # The last row is junk and needs to be cut
    parse_dates=["Plant Date"],
    dtype={
        "Ward": str
    }
)

Clean up the columns

In [6]:
dss_columns = {
    "WorkOrderId": "id",
    "Plant Date": "date",
    "Address": "address",
    "Description": "description",
    "Ward": "ward",
    "APS:": "trees",
}

In [7]:
trimmed_dss_df = (
    dss_df[dss_columns.keys()]
    .rename(columns=dss_columns)
    .sort_values("date")
)

Tag the source

In [8]:
trimmed_dss_df['source'] = 'dss'

Filter it down to tree plants, excluding other types of actions

In [9]:
filtered_dss_df = (
    trimmed_dss_df[trimmed_dss_df.description == 'SED_Tree Planting']
    .drop('description', axis=1)
)

Remove duplicate trees using the identifier field

In [10]:
deduped_dss_df = filtered_dss_df.drop_duplicates("id", keep="first")

## Department of Transportation

Read in the two raw data files

In [11]:
cdot_df = pd.read_csv(
    "input/parsed-01.01.2008_12.31.2020311CSR.1.csv",
    parse_dates=["created_date", "activity_date"]
)

In [12]:
cdot2_df = pd.read_csv(
    "input/tabula-11.30.2018-12.31.2020311CSR.2.csv",
    dtype={
        "Ward": str
    }
)

Create a date field based on a fallback scheme. If the activity date is there, use that. If not, use the created date.

In [13]:
cdot_df['date'] = cdot_df.activity_date.mask(pd.isnull, cdot_df['created_date'])

Clean up the columns

In [14]:
cdot_columns = {
    "sr_number": "id",
    "date": "date",
    "address": "address",
    "ward": "ward"
}

In [15]:
trimmed_cdot_df = (
    cdot_df[cdot_columns.keys()]
    .rename(columns=cdot_columns)
    .sort_values("date")
)

Clean up the ward field so it only contains a number

In [16]:
trimmed_cdot_df.ward = trimmed_cdot_df.ward.apply(lambda x: x.replace("-Ward", "").strip() if not pd.isnull(x) else pd.NA)

In [17]:
trimmed_cdot_df.loc[
    trimmed_cdot_df["ward"] == "0",
    'ward'
] = pd.NA

Since each row represents a single tree, set the value as 1.

In [18]:
trimmed_cdot_df['trees'] = 1

Tag the source

In [19]:
trimmed_cdot_df['source'] = 'cdot'

Get rid of really old dates

In [20]:
filtered_cdot_df = trimmed_cdot_df[trimmed_cdot_df.date >= "2008-01-01"]

Calculate an address column for CDOT's second sheet.

In [21]:
cdot2_df['address'] = cdot2_df['Work\nOrder:\nStreet\nDirection'] + " " + cdot2_df['Work Order:\nStreet Name'] + " " + cdot2_df['Work\nOrder:\nStreet Type']

Clean up the columns

In [22]:
cdot2_columns = {
    "Work Order Number": "id",
    "address": "address",
    "Ward": "ward"
}

In [23]:
trimmed_cdot2_df = (
    cdot2_df[cdot2_columns.keys()]
    .rename(columns=cdot2_columns)
)

Set the tree number again.

In [24]:
trimmed_cdot2_df['trees'] = 1

Tag the source again

In [25]:
trimmed_cdot2_df['source'] = 'cdot'

## Parks Department

Read in the raw data

In [26]:
parks_df = pd.read_csv(
    "input/tabula-Park District Tree Plantings.csv"
)

Standardize the columns

In [27]:
parks_columns = {
    "Park": "address",
    "Tree Total": "trees",
    "Year": "year"
}

In [28]:
trimmed_parks_df = (
    parks_df[parks_columns.keys()]
    .rename(columns=parks_columns)
)

Tag the source

In [29]:
trimmed_parks_df['source'] = 'parks'

Get rid of junky old dates

In [30]:
filtered_parks_df = trimmed_parks_df[trimmed_parks_df.year >= 2008]

## Concatenate

Combine all the dataframes together

In [32]:
concat_df = pd.concat([deduped_dss_df, filtered_cdot_df, trimmed_cdot2_df, filtered_parks_df])

Add a year column

In [33]:
concat_df['year'] = concat_df['date'].dt.year

## Export

Output the results

In [34]:
concat_df.to_csv("./output/trees.csv", index=False)